In [1]:
#laos data viz!

In [2]:
#path configuration
to_data_parent = "../USAID-Laos-EV" #path to the parent folder, should contain program subfolders
to_data_folder = "../USAID-Laos-EV" #data folder, where composite data files will be written/read

program = "usaid_laos_ev"

In [3]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

params = {'legend.fontsize': 'small',
          'figure.figsize': (10, 8),
         'axes.labelsize': 'small',
         'axes.titlesize':'small',
         'xtick.labelsize':'small',
         'ytick.labelsize':'small'}
plt.rcParams.update(params)


import importlib

In [ ]:
print('starting with ', program)

#create dataset with surveys and trips
print(to_data_parent + '/' + program + '_trips.csv')
trips = pd.read_csv(to_data_parent + '/' + program + '_trips.csv')
print(len(trips), 'trips')
print(trips.user_id.nunique(), 'people')

surveys = pd.read_csv(to_data_parent + '/' + program + '_demographics.csv')
print(len(surveys), 'surveys')

#drop any null ids
socio_data = surveys[~surveys['user_id'].isnull()]
print(len(socio_data), 'surveys after dropping null ids')

#drop duplicates
socio_data = socio_data.sort_values(by=['user_id', 'data.ts'])
socio_data.drop_duplicates(subset=['user_id'], keep='last', inplace=True)
print(len(socio_data),'surveys', socio_data['user_id'].nunique(), 'users after dropping duplicates')

#prepare survey ids for merging
socio_data['user_id_socio'] = socio_data['user_id'].astype(str)
socio_data['user_id_socio'] = socio_data['user_id_socio'].str.strip() #remove leading or trailing whitespace!!
socio_data['user_id_socio'] = [i.replace('-','') for i in socio_data.user_id_socio]
socio_data['user_id_socio'] = socio_data['user_id_socio']
socio_data = socio_data.drop(labels='user_id', axis=1)


#prepare trip ids for merging
trips['user_id_socio'] = trips.user_id.astype(str)
trips['user_id_socio'] = trips['user_id_socio'].str.strip() #remove leading or trailing whitespace!!
trips.user_id_socio = [i.replace('-','') for i in trips.user_id_socio] # remove all dashes from strings

#merge the data
data = trips.merge(socio_data, on='user_id_socio')
print(len(data), 'trips after merging')
print(data.user_id_socio.nunique(), 'people after merging')

data['program'] = program

In [ ]:
#merge them all together
full_data = data.copy()
print(len(full_data), 'trips')
print(full_data.user_id.nunique(), 'users')

full_data.to_csv(to_data_folder + "/expanded_ct.csv")

In [ ]:
# loading the data
data = pd.read_csv(to_data_folder + "/expanded_ct.csv")
data.columns

In [ ]:
# Summary statistics table
print(len(pd.unique(data.user_id)))
stat_data = data[['data.distance','data.duration']]
stat_data.describe()

In [ ]:
#format the age datapoints for the chart
data = data.replace(['_16_years_old',
              '16___20_years_old',
              '21___25_years_old', 
              '26___30_years_old', 
              '31___35_years_old',
              '36___40_years_old',
              '41___45_years_old', 
              '56___60_years_old'], 
             ['<16',
              '16-20',
              '21-25',
              '26-30',
              '31-35',
              '36-40',
              '41-45',
              '56-60'])

In [ ]:
# Age, Income, Gender
plot_data = data.copy()
plot_data = plot_data.groupby(['user_id']).nth(0)[['How_old_are_you','What_is_your_gender']].dropna()
plot_data = plot_data[plot_data['What_is_your_gender'].isin(['man','woman'])]
plot_data = plot_data.groupby(['user_id'], as_index=False).nth(0)

print(len(plot_data)) #22 participants

#order the values
plot_data['How_old_are_you'] = pd.Categorical(plot_data['How_old_are_you'], [ '<16',
                                                                              '16-20',
                                                                              '21-25',
                                                                              '26-30',
                                                                              '31-35',
                                                                              '36-40',
                                                                              '41-45',
                                                                              '56-60'])

In [ ]:
plot_title='Participant Demographics'
ylab='Count'
file_name='CanBikeCO_report_demog'

fig, axs = plt.subplots(2,2,figsize=(10,6))
sns.histplot(data=plot_data, x='What_is_your_gender', ax=axs[0,0], color='purple', stat='probability').set(xlabel='Sex',ylabel='proportion')
sns.histplot(data=plot_data, x='How_old_are_you', ax=axs[0,1], color='red', stat='probability').set(xlabel='Age',ylabel='proportion')
plt.xticks(rotation=35, ha='right')
plt.tight_layout()

fig.savefig(file_name+".png", bbox_inches='tight')

In [ ]:
#gender - mode correlation
mode_gender_data=data.copy()
mode_gender_data = mode_gender_data[['What_is_your_gender', 'mode_confirm', 'data.distance']]
mode_gender_data = mode_gender_data[mode_gender_data.mode_confirm.notna()]

print(len(mode_gender_data))
mode_gender_data['What_is_your_gender'] = mode_gender_data['What_is_your_gender'].astype('category')
mode_gender_data['mode_confirm'] = mode_gender_data['mode_confirm'].astype('category')

gender_man = mode_gender_data[mode_gender_data['What_is_your_gender'] == 'man']
print(len(gender_man))
gender_woman = mode_gender_data[mode_gender_data['What_is_your_gender'] == 'woman']
print(len(gender_woman))

t1 = gender_man.groupby(['mode_confirm'], as_index=False).count()[['mode_confirm','data.distance']]
t1['proportion'] = t1['data.distance'] / np.sum(t1['data.distance'])
t1['trip_type'] = 'Man'
t1.columns = ['Mode','Count','Proportion','Gender']

# processing long data
t2 = gender_woman.groupby(['mode_confirm'], as_index=False).count()[['mode_confirm','data.distance']]
t2['proportion'] = t2['data.distance'] / np.sum(t2['data.distance'])
t2['trip_type'] = 'Woman'
t2.columns = ['Mode','Count','Proportion','Gender']

plot_data = pd.concat([t1, t2])
plot_data = plot_data.reset_index()
plot_data = plot_data.drop('index', axis='columns')

plot_data

In [ ]:
## code from minipilot vs full pilot regular / commute trip split -- has better labels!!

width = 0.8
fig, ax = plt.subplots(figsize=(10, 6))
running_total = [0,0]
fig_data = plot_data.copy()

for mode in pd.unique(fig_data.Mode):
    band_data = fig_data[fig_data['Mode']==mode]
    
    labels = band_data['Gender']
    vals = band_data['Proportion']*100
    bar_labels = band_data['Count']
    
    vals_str = [f'{y:.1f} %\n({x:,})' if y>10 else '' for x, y in zip(bar_labels, vals)]
    bar = ax.barh(labels, vals, width, left=running_total, label=mode)
    ax.bar_label(bar, label_type='center', labels=vals_str, rotation=90, fontsize=18)
    running_total[0] = running_total[0]+vals.iloc[0]
    running_total[1] = running_total[1]+vals.iloc[1]

ax.set_title('Mode Distribution', fontsize=25)
ax.legend(bbox_to_anchor=(1,1), fancybox=True, shadow=True, fontsize=12)
plt.subplots_adjust(bottom=0.20)
fig.tight_layout()
plt.show()

In [ ]:
##vertical chart

test_data = plot_data.drop('Count', axis='columns')
data_man = test_data[test_data['Gender']=='Man']
data_woman = test_data[test_data['Gender']=='Woman']

data_man = data_man.drop('Gender', axis='columns')
data_man.columns = ['Mode','Man']
data_man = data_man.set_index('Mode')
data_man = data_man.transpose()

data_woman = data_woman.drop('Gender', axis='columns')
data_woman.columns = ['Mode','Woman']
data_woman = data_woman.set_index('Mode')
data_woman = data_woman.transpose()

test_data = pd.concat([data_man, data_woman])
test_data = test_data.fillna(0)


test_data

ax = test_data.plot.bar(stacked=True, 
                        title="Mode Share by Gender", 
                        ylabel="Proportion of Total Trip Count (%)",
                        xlabel = "Participant Gender",
                        rot = 0)

for c in ax.containers:
    labels = [f'{round(v.get_height()*100)}%' if v.get_height() > .05 else ''for v in c]
    ax.bar_label(c, labels=labels, label_type='center')

ax.set_xticklabels(test_data.index, rotation=0, ha='center')
ax.legend(bbox_to_anchor=(1,1), fancybox=True, shadow=True, fontsize=18)

ax.set_xlabel("Participant Gender", fontsize = 18)
ax.set_ylabel("Proportion of Total Trip Count (%)", fontsize = 18)
ax.set_title("Mode Share by Gender", fontsize = 22)

plt.savefig("CanBikeCO_report_age_mode_share.jpeg", bbox_inches='tight')


##this would be better if we also had the counts!!!

In [ ]:
plot_data = data.copy()
plot_data = plot_data[plot_data['data.distance']<20]

plot_title = 'Mode Share by Trip Distance'
ylab = 'Total Trips'
file_name = "CanBikeCO_report_mode_share_distance_short"
sns.histplot(plot_data, x="data.distance", hue="mode_confirm", element="poly", multiple="stack")

In [ ]:
plot_data = data.copy()
plot_data = plot_data[plot_data['data.distance']<6]

plot_title = 'Mode Share by Trip Distance'
ylab = 'Total Trips'
file_name = "CanBikeCO_report_mode_share_distance_short"
sns.histplot(plot_data, x="data.distance", hue="mode_confirm", element="poly", multiple="stack")

In [ ]:
## DISTANCE BY MODE -- WANT 1 for Man and 1 for Woman!

# Distribution of distances by program
plot_data = data.copy()

#get rid of planes - outliers!
plot_data = plot_data[plot_data['mode_confirm'] != 'air']
plot_data = plot_data[plot_data['mode_confirm'] != 'taxi']
plot_data = plot_data[plot_data['mode_confirm'] != 'auto_rickshaw']

plot_title = 'Distribution of Distances by Mode'
ylab = 'Distance (miles)'

fig, ax = plt.subplots(figsize=(10,8))
sns.boxplot(ax=ax, data=plot_data, x='mode_confirm', y='data.distance', hue='mode_confirm', showfliers=False).set(title=plot_title, xlabel='', ylabel=ylab)
plt.subplots_adjust(bottom=0.25)
plt.xticks(rotation=35, ha='right')
plt.legend([])

In [ ]:
#experimenting with the trajectories

In [5]:
print('starting with ', program)

#create dataset with surveys and trips
# print(to_data_parent + '/' + program + '_trips.csv')
# trips = pd.read_csv(to_data_parent + '/' + program + '_trips.csv')
# print(len(trips), 'trips')
# print(trips.user_id.nunique(), 'people')

surveys = pd.read_csv(to_data_parent + '/' + program + '_demographics.csv')
print(len(surveys), 'surveys')

trajs = pd.read_csv(to_data_parent + '/' + program + '_trajectories.csv')
print(len(trajs), 'trajectories')

#drop any null ids
socio_data = surveys[~surveys['user_id'].isnull()]
print(len(socio_data), 'surveys after dropping null ids')

#drop duplicates
socio_data = socio_data.sort_values(by=['user_id', 'data.ts'])
socio_data.drop_duplicates(subset=['user_id'], keep='last', inplace=True)
print(len(socio_data),'surveys', socio_data['user_id'].nunique(), 'users after dropping duplicates')

#prepare survey ids for merging
socio_data['user_id_socio'] = socio_data['user_id'].astype(str)
socio_data['user_id_socio'] = socio_data['user_id_socio'].str.strip() #remove leading or trailing whitespace!!
socio_data['user_id_socio'] = [i.replace('-','') for i in socio_data.user_id_socio]
socio_data['user_id_socio'] = socio_data['user_id_socio']
socio_data = socio_data.drop(labels='user_id', axis=1)

# #prepare trip ids for merging
# trips['user_id_socio'] = trips.user_id.astype(str)
# trips['user_id_socio'] = trips['user_id_socio'].str.strip() #remove leading or trailing whitespace!!
# trips.user_id_socio = [i.replace('-','') for i in trips.user_id_socio] # remove all dashes from strings

#prepare traj ids for merging
trajs['user_id_socio'] = trajs.user_id.astype(str)
trajs['user_id_socio'] = trajs['user_id_socio'].str.strip() #remove leading or trailing whitespace!!
trajs.user_id_socio = [i.replace('-','') for i in trajs.user_id_socio] # remove all dashes from strings

# #merge the data
# data = trips.merge(socio_data, on='user_id_socio')
# print(len(data), 'trips after merging')
# print(data.user_id_socio.nunique(), 'people after merging')

#merge the trajectories
traj_data = trajs.merge(socio_data, on='user_id_socio')
print(len(traj_data), 'trajs after merging')
print(traj_data.user_id_socio.nunique(), 'people after merging')

traj_data['program'] = program

starting with  usaid_laos_ev
28 surveys
250000 trajectories
28 surveys after dropping null ids
25 surveys 25 users after dropping duplicates
233050 trajs after merging
19 people after merging


In [ ]:
traj_data.columns

In [ ]:
traj_data['data.distance'].max()

In [ ]:
plot_data = traj_data.copy()
plot_data = plot_data[plot_data['data.distance']<200]

plot_title = 'Distribution of Distances by Mode'
ylab = 'Distance (meters?)'

fig, ax = plt.subplots(figsize=(10,8))
sns.boxplot(ax=ax, data=plot_data, x='data.mode_str', y='data.distance', hue='data.mode_str', showfliers=False).set(title=plot_title, xlabel='', ylabel=ylab)
plt.subplots_adjust(bottom=0.25)
plt.xticks(rotation=35, ha='right')
plt.legend([])

#not really seeming reliable or significant ... outliers? 

In [7]:
plot_data = traj_data.copy()

In [8]:
plot_data.columns

Index(['_id_x', 'user_id', 'data.latitude', 'data.longitude', 'data.ts_x',
       'data.fmt_time_x', 'data.altitude', 'data.distance', 'data.speed',
       'data.heading', 'data.idx', 'data.mode', 'data.section', 'data.filter',
       'data.floor', 'data.sensed_speed', 'data.vaccuracy', 'data.accuracy',
       'data.elapsedRealtimeNanos', 'data.mode_str', 'user_id_socio', '_id_y',
       'data.ts_y', 'At_your_primary_job_do_you_ha',
       'Which_best_describes_your_prim', 'Do_you_work_full_time_or_part_',
       'Do_you_have_the_option_of_work', 'Please_describe_your_primary_job',
       'Do_you_have_more_than_one_job', 'What_days_of_the_week_do_you_t',
       'How_many_days_do_you_usually_w_001', 'Which_one_below_describe_you_b',
       'What_is_your_race_ethnicity', 'Are_you_a_student',
       'What_is_the_highest_grade_or_d', 'do_you_consider_yourself_to_be',
       'What_is_your_gender', 'How_old_are_you', 'Are_you_a_paid_worker',
       'Do_you_have_a_driver_license', 'How_long_y

In [14]:
plot_data['date'] = str(plot_data['data.local_dt.year']) + str(plot_data['data.local_dt.month']) + str(plot_data['data.local_dt.day'])

In [17]:
plot_data['date'].unique()

array(['0         2023\n1         2023\n2         2023\n3         2023\n4         2023\n          ... \n233045    2023\n233046    2023\n233047    2023\n233048    2023\n233049    2023\nName: data.local_dt.year, Length: 233050, dtype: int640         12\n1         12\n2         12\n3         12\n4         12\n          ..\n233045    12\n233046    12\n233047    12\n233048    12\n233049    12\nName: data.local_dt.month, Length: 233050, dtype: int640         5\n1         5\n2         5\n3         5\n4         5\n         ..\n233045    5\n233046    5\n233047    5\n233048    5\n233049    5\nName: data.local_dt.day, Length: 233050, dtype: int64'],
      dtype=object)